In [1]:
import os
import pandas as pd
import csv
import numpy as np
from joblib import dump, load
import h5py

from matplotlib import pyplot as plt
import mpl_toolkits.mplot3d.axes3d as p3
from matplotlib import animation
%matplotlib notebook

import cv2
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas

# Gather instances

In [2]:
experiment_path = "../predictions/20201119/bottomup/pred_FishTank20200416_160648/results/FishTank20200416_160648/"
video_path = "../../../zebrafish_labeling_GUI/OneFish_20200416/"
cameras = ["D_xz", "E_xy", "F_yz"]

In [5]:
print(experiment_path.split('/')[-2])

FishTank20200416_160648


In [3]:
splitdatas = sorted([file.split('.')[0] for file in os.listdir(video_path + cameras[0]) if file.split('0')[0] == 'splitdata'])
print(splitdatas)

['splitdata0007', 'splitdata0008', 'splitdata0009', 'splitdata0010', 'splitdata0011', 'splitdata0012', 'splitdata0013', 'splitdata0014', 'splitdata0015', 'splitdata0016', 'splitdata0017', 'splitdata0018', 'splitdata0019', 'splitdata0020', 'splitdata0021', 'splitdata0022', 'splitdata0023', 'splitdata0024', 'splitdata0025', 'splitdata0026', 'splitdata0027', 'splitdata0028', 'splitdata0029', 'splitdata0030', 'splitdata0031', 'splitdata0032', 'splitdata0033', 'splitdata0034', 'splitdata0035', 'splitdata0036', 'splitdata0037', 'splitdata0038', 'splitdata0039', 'splitdata0040', 'splitdata0041', 'splitdata0042', 'splitdata0043', 'splitdata0044', 'splitdata0045', 'splitdata0046', 'splitdata0047', 'splitdata0048', 'splitdata0049', 'splitdata0050', 'splitdata0051', 'splitdata0052', 'splitdata0053', 'splitdata0054', 'splitdata0055', 'splitdata0056', 'splitdata0057', 'splitdata0058', 'splitdata0059', 'splitdata0060', 'splitdata0061', 'splitdata0062', 'splitdata0063', 'splitdata0064', 'splitdata006

In [7]:
# get total frames
max_values = []
    
for split in splitdatas:
    max_frames_split = 0
    for cam in cameras:
        cam_path = experiment_path + cam + '/' + split + '.csv'
        csv_file = pd.read_csv(cam_path, header=None)
        if not csv_file[1].all() == 0:
            print("tracking error found!")
        if max(csv_file[0]) > max_frames_split:
            max_frames_split = max(csv_file[0])
    max_values.append(max_frames_split + 1)
    if max_frames_split + 1 < 6000:
        print(split)
print(max_values)

splitdata0102
[6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 3978]


In [12]:
# gather all original_instances
original_instances = np.zeros(shape=(3, sum(max_values), 1, 3, 2))

for i, cam in enumerate(cameras):
    frames_done = 0
    for j, split in enumerate(splitdatas):
        cam_path = experiment_path + cam + '/' + split + '.csv'
        with open(cam_path) as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')
            for row in csv_reader:
                frame = int(row[0])
                part = int(row[2])
                original_instances[i, frame + sum(max_values[0:j]), 0, part, 0] = float(row[3]) 
                original_instances[i, frame + sum(max_values[0:j]), 0, part, 1] = float(row[4])

In [13]:
# convert zeros to NaNs
original_instances = original_instances.astype('float')
original_instances[original_instances == 0] = np.NaN

# save
original_instances_path = experiment_path + experiment_path.split('/')[-2] + ".h5"
with h5py.File(original_instances_path, 'w') as hf:
    hf.create_dataset('original_instances', data=original_instances)

# Calculate 3D values

In [14]:
class Calibration(object):
    ''' A class for loading and using the calibrations for the cameras
    
    -- Methods --
    compute_imageCoord_triplet_from_XYZ
    compute_XYZ_from_imageCoord_triplet
    compute_XZ_imcoords_from_XY_YZ
    compute_XY_imcoords_from_XZ_YZ
    compute_YZ_imcoords_from_XZ_XY
    compute_point_correspondence_error
    
    '''
    
    def __init__(self, calibration_folder_path):
        ''' Instantiate the object
        
        -- args --
        calibration_folder_path: the path to a calibration folder, where the regressed functions
                                 have already been computed and saved
                                 
        '''
        # record the folder paths
        self.calibration_folder_path = calibration_folder_path
        self.python_calibration_folderPath = os.path.join(self.calibration_folder_path, 
                                                          'python_calibration_models')
        
        # load the models and assign as attributes
        self._load_models()
        
    
    def _load_models(self):
        ''' Instantiate the regression object attributes:
            xyz_getter, imCoord_getter, xz_getter, xy_getter, yz_getter 
        '''
        imCoords_to_XYZ_path = os.path.join(self.python_calibration_folderPath, 'imCoords_to_XYZ.joblib')
        XYZ_to_imCoords_path = os.path.join(self.python_calibration_folderPath, 'XYZ_to_imCoords.joblib')
        xy_yz_to_xz_path = os.path.join(self.python_calibration_folderPath, 'xy_yz_to_xz.joblib')
        xz_yz_to_xy_path = os.path.join(self.python_calibration_folderPath, 'xz_yz_to_xy.joblib')
        xz_xy_to_yz_path = os.path.join(self.python_calibration_folderPath, 'xz_xy_to_yz.joblib')
        
        self.xyz_getter = load(imCoords_to_XYZ_path)
        self.imCoord_getter = load(XYZ_to_imCoords_path)
        self.xz_getter = load(xy_yz_to_xz_path)
        self.xy_getter = load(xz_yz_to_xy_path)
        self.yz_getter = load(xz_xy_to_yz_path)
        return
    
    
    # ---- Main Methods ---- #
    
    def compute_imageCoord_triplet_from_XYZ(self, XYZ):
        ''' Predict the image coordinates in all 3 camera views of the
            3D point XYZ
            
        -- inputs --
        XYZ: array (3,), the position of a point in 3D
        
        -- returns --
        imCoords: array (3,2) of image coordinates in standard camera
                  order of XZ,XY,YZ
        '''
        imCoords = self.imCoord_getter.predict(XYZ.reshape(1,-1))
        imCoords = imCoords.reshape(3,2)
        return imCoords
    
    
    def compute_XYZ_from_imageCoord_triplet(self, imCoords):
        ''' Predict the XYZ position of the point given by the image
            coordinates from all 3 cameras

        -- Inputs --
        imCoords: array of shape (3,2)

        -- Outputs --
        XYZ: array of shape (3)

        '''
        XYZ = self.xyz_getter.predict(imCoords.reshape(-1,6))
        return XYZ
    
    
    def compute_XZ_imcoords_from_XY_YZ(self, xy_imCoord, yz_imCoord):
        ''' Given an image coordinate from both the XY and YZ views,
            compute the corresponding image coordinate from the XZ view

        -- args --
        xy_imCoord: image coordinate of shape (2,)
        yz_imCoord: image coordinate of shape (2,)

        -- returns --
        xz_imCoord: image coordinate of shape (2,)

        '''
        input_data = np.hstack((xy_imCoord, yz_imCoord)).reshape(1,4)
        xz_imCoord = self.xz_getter.predict(input_data)
        return xz_imCoord

    def compute_XY_imcoords_from_XZ_YZ(self, xz_imCoord, yz_imCoord):
        ''' Given an image coordinate from both the XZ and YZ views,
            compute the corresponding image coordinate from the XY view

        -- args --
        xz_imCoord: image coordinate of shape (2,)
        yz_imCoord: image coordinate of shape (2,)
        
        -- returns --
        xy_imCoord: image coordinate of shape (2,)
        '''
        # prepare the input for predictor, and predict the imcoord
        input_data = np.hstack((xz_imCoord, yz_imCoord)).reshape(1,4)
        xy_imCoord = self.xy_getter.predict(input_data)
        return xy_imCoord

    def compute_YZ_imcoords_from_XZ_XY(self, xz_imCoord, xy_imCoord):
        ''' Given an image coordinate from both the XY and YZ views,
            compute the corresponding image coordinate from the XZ view

        -- args --
        xz_imCoord: image coordinate of shape (2,)
        xy_imCoord: image coordinate of shape (2,)

        -- returns --
        yz_imCoord: image coordinate of shape (2,)

        '''
        # prepare the input for predictor, and predict the imcoord
        input_data = np.hstack((xz_imCoord, xy_imCoord)).reshape(1,4)
        yz_imCoord = self.yz_getter.predict(input_data)
        return yz_imCoord
    
    
    def compute_point_correspondence_error(self, camIdxs, imCoords_cam1, imCoords_cam2):
        ''' Compute the error of making a cross-camera association between these points

        -- args -- 
        camIdxs: a list denoting the cameras the imCoords args are coming from.
                 Has to be [0,1], [1,2], or [0, 2]
        imCoords_cam1: image coordinates from a camera
        imCoords_cam2: image coordinates from a different camera


        -- returns --
        error: a scalar error value for making this association 
        '''
        # STEP 0: The error is NaN if either point is NaN
        if np.all(np.isnan(imCoords_cam1)) or np.all(np.isnan(imCoords_cam2)):
            return np.NaN

        # STEP 1: Compute the proposed image coordinate triplet
        if camIdxs == [0,1]:
            # derive YZ
            imCoords_cam3 = self.compute_YZ_imcoords_from_XZ_XY(imCoords_cam1, imCoords_cam2)
            proposed_imCoords = np.vstack((imCoords_cam1, imCoords_cam2, imCoords_cam3))
        elif camIdxs == [0, 2]:
            # derive XY
            imCoords_cam3 = self.compute_XY_imcoords_from_XZ_YZ(imCoords_cam1, imCoords_cam2)
            proposed_imCoords = np.vstack((imCoords_cam1, imCoords_cam3, imCoords_cam2))
        elif camIdxs == [1, 2]:
            # derive XZ
            imCoords_cam3 = self.compute_XZ_imcoords_from_XY_YZ(imCoords_cam1, imCoords_cam2)
            proposed_imCoords = np.vstack((imCoords_cam3, imCoords_cam1, imCoords_cam2))


        # STEP 2: Compute the errors

        # For each pairing of cameras, compute the 3rd cam image coordinate,
        # then compare this triplet to the proposed_imCoords, which act as truth
        # Note1: If this is a good pairing, then proposed_imCoords represent the same point in 3D
        # Note2: for one of these camera pairings test, we will get back an error of 0,
        #        since we did the same computation to compute proposed_coordinates.
        # Note3: to deal with note2, we define the error as the maximum of the 3 errors
        derived_xz = self.compute_XZ_imcoords_from_XY_YZ(proposed_imCoords[1], proposed_imCoords[2])
        image_coords_derXZ = np.vstack((derived_xz, proposed_imCoords[1], proposed_imCoords[2]))
        error_derXZ = np.linalg.norm(proposed_imCoords - image_coords_derXZ)

        derived_xy = self.compute_XY_imcoords_from_XZ_YZ(proposed_imCoords[0], proposed_imCoords[2])
        image_coords_derXY = np.vstack((proposed_imCoords[0], derived_xy, proposed_imCoords[2]))
        error_derXY = np.linalg.norm(proposed_imCoords - image_coords_derXY)

        derived_yz = self.compute_YZ_imcoords_from_XZ_XY(proposed_imCoords[0], proposed_imCoords[1])
        image_coords_derYZ = np.vstack((proposed_imCoords[0], proposed_imCoords[1], derived_yz))
        error_derYZ = np.linalg.norm(proposed_imCoords - image_coords_derYZ)

        errors = np.vstack((error_derXY, error_derXY, error_derYZ))
        error = np.sum(errors)

        return error

In [15]:
# calibration folder
calibrationFolderPath = '/home/thomasreus/Documents/Project/LargeCrop/20200325_calibration/shared_20200907/'

# set other information of importance
padding_depth = 70
numFish = 1
numBodyPoints = 3
numCams = 3

# calibration object
cal = Calibration(calibrationFolderPath)

instances = np.copy(original_instances)

In [16]:
# loop over data and get 3D points
numFrames = len(instances[0])
positions_3D = np.zeros((numFrames, 1, numBodyPoints, 3))
undefined_bps = {}
xz_fail = 0
xy_fail = 0
yz_fail = 0

for frame in range(numFrames):
    
    for bpIdx in range(numBodyPoints):
        
        # bodypoint coordinates
        bpCoordTriplet = instances[:, frame, 0, bpIdx]
        
        # check for NaN
        if np.isnan(bpCoordTriplet).any():
            validCams = []
            invalidCams = []
            for cam, coords in enumerate(bpCoordTriplet):
                if not np.isnan(coords).any():
                    validCams.append(cam)
                else:
                    invalidCams.append(cam)
                    
            # calculate missing data
            if validCams == [0, 1]:
                bpCoordTriplet[2] = cal.compute_YZ_imcoords_from_XZ_XY(bpCoordTriplet[0], bpCoordTriplet[1])
            elif validCams == [0, 2]:
                bpCoordTriplet[1] = cal.compute_XY_imcoords_from_XZ_YZ(bpCoordTriplet[0], bpCoordTriplet[2])
            elif validCams == [1, 2]:
                bpCoordTriplet[0] = cal.compute_XZ_imcoords_from_XY_YZ(bpCoordTriplet[1], bpCoordTriplet[2])
            else:
                for invCam in invalidCams:
                    if invCam == 0:
                        xz_fail += 1
                    elif invCam == 1:
                        xy_fail += 1
                    elif invCam == 2:
                        yz_fail += 1
                try:
                    undefined_bps[frame].append([bpIdx, invalidCams])
                except:
                    undefined_bps[frame] = [[bpIdx, invalidCams]]
                continue
                
        # compute 3D from data
        bp_3D = cal.compute_XYZ_from_imageCoord_triplet(bpCoordTriplet)
        positions_3D[frame, 0, bpIdx] = bp_3D

In [17]:
print(len(undefined_bps))

173


In [18]:
print(xz_fail, xy_fail, yz_fail)

152 272 351


In [19]:
# convert zeros to NaNs
positions_3D = positions_3D.astype('float')
positions_3D[positions_3D == 0] = np.NaN

# positions_3D[positions_3D == 0] = np.NaN
with h5py.File(original_instances_path, 'a') as hf:
    hf.create_dataset('tracks_3D', data=positions_3D)

In [20]:
print(positions_3D.shape)

(573978, 1, 3, 3)


# Make 3D animation

In [ ]:
def make_frame(data_frame):
    plt.ioff()
    fig = plt.figure(figsize=(10,10))
    canvas = FigureCanvas(fig)
    fig.add_subplot(111, projection='3d')
    ax = fig.gca()
    c_real = [0,0,1,1]
    
    ax.scatter(data_frame[0, 0], data_frame[0, 1], data_frame[0, 2], color=c_real, marker='D', depthshade=False, s=40)
    ax.scatter(data_frame[1, 0], data_frame[1, 1], data_frame[1, 2], color=c_real, marker='x', depthshade=False, s=40)
    ax.scatter(data_frame[2, 0], data_frame[2, 1], data_frame[2, 2], color=c_real, marker='.', depthshade=False, s=40)
    ax.plot3D(data_frame[:, 0], data_frame[:, 1], data_frame[:, 2], color=c_real, linewidth=2)
    
    ax.set_xlabel('X (cm)', fontsize=20)
    ax.set_ylabel('Y (cm)', fontsize=20)
    ax.set_zlabel('Z (cm)', fontsize=20)
    ax.set_xlim(0,40)
    ax.set_ylim(0,40)
    ax.set_zlim(0,40)
    ticks = [0, 10, 20, 30, 40]
    ax.set_xticks(ticks, minor=False)
    ax.set_yticks(ticks, minor=False)
    elevation_angle = 10
    azimuthal_angle = 45
    ax.view_init(elevation_angle, azimuthal_angle)
    
    canvas.draw()
    s, (width, height) = canvas.print_to_buffer()
    
    # convert the figure to an array of the correct size
    img = np.frombuffer(canvas.tostring_rgb(), dtype='uint8').reshape(width, height, 3)
    
    plt.close()
    plt.ion()
    
    return img

In [ ]:
coords = np.copy(positions_3D)

# pick a start and stop frame
startFrame = 0
stopFrame = 60000

# set other parameters for the movie
movPath = experiment_path + '10minute.mp4'
fps = 100
fourcc = cv2.VideoWriter_fourcc('m','p','4','v')
resolution = (500,500) 
out = cv2.VideoWriter(movPath, fourcc, fps, resolution)

for n in range(startFrame, stopFrame):
    if np.mod(n, 500) == 0:
        print(n)
    
    # Make sLEAP panel
    sLEAP_panel = make_frame(coords[n, 0])
    
    # resize the panel
    sLEAP_panel = cv2.resize(sLEAP_panel, (500,500), interpolation=cv2.INTER_AREA)
    
    # draw the frame number
    font                   = cv2.FONT_HERSHEY_SIMPLEX
    bottomLeftCornerOfText = (10,480)
    fontScale              = 1
    fontColor              = (0,0,0)
    lineType               = 2
    cv2.putText(sLEAP_panel,str(n).zfill(6), 
        bottomLeftCornerOfText, 
        font, 
        fontScale,
        fontColor,
        lineType)
    
    # write the frame
    out.write(sLEAP_panel)

# close the writer
out.release()